# Create covariate stacks

### Covariates will be derived by reducing satellite imagery to useful modelling layers
#### Covariates can include: Landsat 8, NASADEM, Sentinel-2, Sentinel-1

### Inputs:
#### bbox_img_tile: An image tile extent will provide the bbox that accesses cloud-optimized GeoTiffs
#### filters: search for imagery based on days-of-year, cloudcover, year range
##### Note: Image tile extent will be dictated by data volume of covars & CPU RAM

In [7]:
# Search for imagery
# https://github.com/developmentseed/example-jupyter-notebooks/blob/landsat-search/notebooks/Landsat8-Search/L8-USGS-satapi.ipynb
import os
import json
import requests
import datetime

sat_api_url = "https://landsatlook.usgs.gov/sat-api"

def query_satapi(query):
    headers = {
            "Content-Type": "application/json",
            "Accept-Encoding": "gzip",
            "Accept": "application/geo+json",
        }

    url = f"{sat_api_url}/stac/search"
    data = requests.post(url, headers=headers, json=query).json()
    
    return data

def query_year(year, bbox, min_cloud, max_cloud):
    '''Given the year, finds the number of scenes matching the query and returns it.'''
    date_min = '-'.join([str(year), "06-01"])
    date_max = '-'.join([str(year), "09-15"])
    start_date = datetime.datetime.strptime(date_min, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(date_max, "%Y-%m-%d") 
    start = start_date.strftime("%Y-%m-%dT00:00:00Z")
    end = end_date.strftime("%Y-%m-%dT23:59:59Z")
    
    query = {
    "time": f"{start}/{end}",
    "query": {
        "bbox":bbox,
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud},
        "collection":{"eq": "landsat-c2l2-sr"}
        },
    "limit": 500 # We limit to 500 items per Page (requests) to make sure sat-api doesn't fail to return big features collection
    }
    
    data = query_satapi(query)
    scenes = data['meta']['found']
    
    return scenes

In [11]:
# Accessing imagery
# Select an area of interest
bbox_list = [[-105,45,-100,50], [-101,45,-100,46]]
min_cloud = 0
max_cloud = 20
for bbox in bbox_list:
    # Geojson of total scenes - Change to list of scenes
    scene_totals = [query_year(year, bbox, min_cloud, max_cloud ) for year in range(2015,2020 + 1)]
    print(scene_totals)

[46585, 45151, 47858, 47872, 48947, 50710]
[46585, 45151, 47858, 47872, 48947, 50710]


In [ ]:
# Processing imagery (band math)
# is this done on the fly?

In [ ]:
# Adding processed imagery to a covariate stack

In [6]:
# Test case for making greenest pixel comps with numpy arrays
import numpy as np
#imagine 3 ndvi layers from 3 seperate images such that

ndvi1 = np.array([[1.2, 1.2, 1.2],[1.2, 2.2, 1.2],[1.2, 1.2, 1.2]])

ndvi2 = np.array([[2.2, 2.2, 2.2],[2.2, 1.2, 2.2],[2.2, 2.2, 2.2]])

ndvi3 = np.array([[2.2, 1.2, 3.2],[1.2, 1.2, 1.2],[2.2, 2.2, 3.2]])

print('ndvi1 = ', ndvi1)
print('ndvi2 = ', ndvi2)
print('ndvi3 = ', ndvi3)

ndvi1 =  [[1.2 1.2 1.2]
 [1.2 2.2 1.2]
 [1.2 1.2 1.2]]
ndvi1 =  [[2.2 2.2 2.2]
 [2.2 1.2 2.2]
 [2.2 2.2 2.2]]
ndvi1 =  [[2.2 1.2 3.2]
 [1.2 1.2 1.2]
 [2.2 2.2 3.2]]


In [2]:
# Now 3 'Red' bands where numbers can be anything
#(but kept the same here for easy tracking, such that:)

Red1 = np.array([[5, 5, 5],[5, 5, 5],[5, 5, 5]])

Red2 = np.array([[6, 6, 6],[6, 6, 6],[6, 6, 6]])

Red3 = np.array([[7, 7, 7],[7, 7, 7],[7, 7, 7]])

print(Red1)
print(Red2)
print(Red3)
# create array of all Green bands

Stack = np.array([Red1, Red2, Red3])

[[5 5 5]
 [5 5 5]
 [5 5 5]]
[[6 6 6]
 [6 6 6]
 [6 6 6]]
[[7 7 7]
 [7 7 7]
 [7 7 7]]


In [3]:
# now, find the index value (position) of the max ndvi layers (based on
# first axis: imagine drilling down through a stack/cube)

max = np.argmax((ndvi1,ndvi2,ndvi3),axis=0)
print(max)

[[1 1 2]
 [1 0 1]
 [1 1 2]]


In [4]:
# Index the Green bands in 'Stack' based on argmax index position
RedComposite = Stack[max,max,max]

In [5]:
print(RedComposite)

[[6 6 7]
 [6 5 6]
 [6 6 7]]
